<center>
    <h1> INF285 - Computación Científica </h1>
    <h2> Tarea 3</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Version: 1.00</h2>
</center>

# Contexto

Para los amantes del vino, la **calidad** es un factor importante al momento de decidir. Las propiedades de un vino, principalmente químicas, influyen en gran medida en el proceso de fermentación. Existen varias carcaterísticas que pueden ayudar a analizar y estudiar la calidad de un vino, ya sea tinto o blanco. A continuación se describen algunas:

* **acidez fija**: tiene que ver con la acidez no volátil del vino.
* **acidez volátil**: es la cantidad de ácido acético en el vino.
* **acidez cítrica**: el ácido cítrico puede agregar sabor al vino.
* **azúcar residual**: es la cantidad de azúcar que queda después del proceso de fermentación.
* **cloruro**: es la cantidad de sal en el vino.
* **dióxido de azufre libre**: cantidad de dióxido de azufre en forma libre.
* **dióxido de azufre total**: cantidad de dióxido de azufre de formas libres y ligadas.
* **densidad**: indica la densidad del vino.
* **ph**: indica el ph del vino.
* **sulfatos**: indica la cantidad de sulfatos en el vino.
* **alcohol**: indica la cantidad porcentual de alcohol en el vino.
* **calidad**: indica la calidad del vino.

Usted cuenta con un archivo denominado *vinos.csv* el cual cuenta en cada columna con los datos de 3000 vinos para cada característica mencionada anteriormente. El problema es que algunas calidades no se han podido obtener, dado que hay **datos faltantes**. Por lo tanto, se necesita aproximar esos datos faltantes para así poder obtener alguna aproximación de la calidad de esos vinos.

**NOTA: cada dato faltante tiene valor -1.0, así como también la calidad del vino que cuenta con un dato faltante.**

# Librerías (no utilizar liberías adicionales)

In [1]:
import numpy as np
import pandas as pd

# Información y funciones entregadas

In [2]:
def obtainData(filename):
    """
    input(s)
    --------
    filename :(str)     filename with format csv.

    output(s)
    ---------
    labels:   (list)    a list with labels.
    mat:      (ndarray) matrix with data of size n x m.
    """
    df = pd.read_csv(filename)
    labels = list(df.columns)
    df = df.drop(columns=["pos"])
    mat = df.to_numpy()
    return labels,mat

In [3]:
labels,mat = obtainData("vinos.csv")

# Consultas

* Enviar a: **tareas.inf285@gmail.com**
* Se recibirán consultas en dos bloques. El primer bloque será desde las 08:00 hrs hasta las 10:00 hrs, donde todas las preguntas recibidas dentro de este bloque se responderán con seguridad a partir de las 10:01 hrs. El segundo bloque de consultas será desde las 10:01 hrs hasta las 14:00 hrs, donde las preguntas recibidas dentro de este bloque horario se responderán con seguridad a partir de las 14:01 hrs.

# Pregunta 1 - Datos Faltantes (20 puntos)

Desarrolle un algoritmo que permita encontrar la fila y la columna respectiva de un dato faltante. La función debe entregar una lista de tuplas, en donde cada tupla *(i,j)* indica el índice de la fila y la columna respectivamente.


In [4]:
def missing_data(mat):
    """
    input(s)
    ----------
    mat:  (ndarray) matrix with information of wines.

    output(s)
    ----------
    index_list: (list) list of tuples with row-column indices of missing data.
    """
    #Aquí va su código

    #calidad no es considerada como dato faltante
    modified_mat = mat[:, :-1]
    index_2D_array = np.where(modified_mat == -1)
    index_list = list(zip(index_2D_array[0], index_2D_array[1]))

    return index_list

# Pregunta 2 (30 puntos): Mínimos Cuadrados I

Desarrolle un algoritmo que permita completar algún dato faltante que se encuentre en la posición $(i,j)$ con las columnas $(j-1)$ y $(j + 1)$. En caso de que el dato faltante se encuentre en la última columna (alcohol), debe utilizar la primera columna como la siguiente (acidez fija). De forma análoga si el dato faltante se encuentra en la primera columna, debe utilizar la última columna como la anterior. El algoritmo recibe como parámetro la posición del dato faltante y la cantidad de valores a utilizar $p$ utilizando mínimos cuadrados. Asumir que se utilizan los $p$ valores anteriores a la fila $i$ y los $p$ valores siguientes a la fila $i$.En este caso el dato faltante se aproxima mediante la función $y = c_1x_1 + c_2x_2$, donde $x_1$ y $x_2$ son las columnas $(j-1)$ y $(j + 1)$ respectivamente. La función debe retornar la aproximación del dato faltante.


In [5]:
#función auxiliar para eliminar la data inválida
def remove_rows_invalid_data(missing_data,cols,y,mat):
    """
    input(s)
    ----------
    missing_data:  (list) list of tuples with row-column indices of missing data.
    cols:          (list) list of column indices where it is necessary to remove data.
    y:             (ndarray) vector that is also modified by the missing data
    mat:           (ndarray) matrix with rows to remove.

    output(s)
    ----------
    modified_mat: (ndarray) matrix without missing data in the columns specified
    modified_y:   (ndarray) vector without missing data in the rows where there is missing data
    """
    #primero se filtran los indices de las columnas que nos interesan
    filtered_missing_indices = [index for index in missing_data if index[1] in cols]

    #sacamos las filas que contienen -1 en alguna de sus columnas
    rows_to_remove = set(index[0] for index in filtered_missing_indices)
    modified_mat = np.delete(mat, list(rows_to_remove), axis=0)
    modified_y = np.delete(y, list(rows_to_remove), axis=0)
    return modified_mat, modified_y


In [6]:
def aprox_lsq(i,j,p,mat):
    """
    input(s)
    ----------
    i:    (int) row index of missing data.
    j:    (int) column index of missing data.
    p:    (int) number of values for approximating the missing data.
    mat:  (ndarray) matrix with information of wines.

    output(s)
    ----------
    y_estimated: (float) missing data value estimated.
    """
    #Aquí va su código

    #la calidad no se considera:
    mat = mat[:, :-1]
    
    col_prev = (j - 1) % mat.shape[1]
    col_next = (j + 1) % mat.shape[1]
  
    start_row = max(i - p, 0)
    end_row = min(i + p + 1, mat.shape[0])

    x1 = np.concatenate((mat[start_row:i, col_prev], mat[i+1:end_row, col_prev]))
    x2 = np.concatenate((mat[start_row:i, col_next], mat[i+1:end_row, col_next]))
    X = np.column_stack((x1, x2))

    y = np.concatenate((mat[start_row:i, j], mat[i+1:end_row, j]))

    X, y= remove_rows_invalid_data(missing_data(X),[col_prev,col_next],y,X)

    Q,R = np.linalg.qr(X, mode="reduced")
    c1,c2 = np.linalg.solve(R, np.dot(Q.T, y))
    
    x1_i_row = mat[i, col_prev] 
    x2_i_row = mat[i, col_next] 

    y_estimated = c1 * x1_i_row + c2 * x2_i_row
    return y_estimated

# Pregunta 3 (40 puntos): Mínimos Cuadrados II.

Suponga que ahora se quiere aproximar los datos faltantes acorde a la siguiente función:

$$
y = a \,\exp(bx)
$$

Desarrolle un algoritmo que aproxime el dato faltante de la posición $(i,j)$ usando la columna anterior $(j-1)$. El algoritmo recibe como parámetro la posición del dato faltante y la cantidad de valores a utilizar $p$ utilizando mínimos cuadrados. Asumir que se utilizan los $p$ valores anteriores a la fila $i$ y los $p$ valores siguientes a la fila $i$.

In [7]:
def aprox_lsq_2(i,j,p,mat):
    """
    input(s)
    ----------
    i:    (int) row index of missing data.
    j:    (int) column index of missing data.
    p:    (int) number of values for approximating the missing data.
    mat:  (ndarray) matrix with information of wines.

    output(s)
    ----------
    y_estimated: (float) missing data value estimated.
    """
    #Aquí va su código
    
    #la calidad no se considera:
    mat = mat[:, :-1]
    col_prev = (j - 1) % mat.shape[1]
  
    start_row = max(i - p, 0)
    end_row = min(i + p + 1, mat.shape[0])
    
    X = np.concatenate((mat[start_row:i, col_prev], mat[i+1:end_row, col_prev]))
    X = np.column_stack([np.ones_like(X),X])
    
    y = np.concatenate((mat[start_row:i, j], mat[i+1:end_row, j]))

    X, y = remove_rows_invalid_data(missing_data(X), col_prev, y, X)
    
    ln_y_values = np.log(y)

    Q,R = np.linalg.qr(X, mode="reduced")
    ln_a, b= np.linalg.solve(R, np.dot(Q.T, ln_y_values))
    x_i_row = mat[i, col_prev]

    y_estimated = np.exp(ln_a + b*x_i_row)
    return y_estimated

In [8]:
aprox_lsq(511,2,100,mat)

0.36155834697257705